In [95]:
from nba_api.stats.endpoints import GameRotation
import pandas as pd

In [96]:
game_rotation = GameRotation(game_id= '0022400062').get_data_frames()
rotation_home, rotation_away = game_rotation[0], game_rotation[1]
rotation = pd.concat([rotation_home, rotation_away], ignore_index=True)

In [97]:
rotation.sort_values('OUT_TIME_REAL')

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
31,0022400062,1610612747,L.A. Lakers,Lakers,2544,LeBron,James,0.0,3160.0,4,4.0,0.333
0,0022400062,1610612750,Minnesota,Timberwolves,201144,Mike,Conley,0.0,3540.0,0,-6.0,0.077
8,0022400062,1610612750,Minnesota,Timberwolves,203944,Julius,Randle,0.0,3540.0,0,-6.0,0.154
39,0022400062,1610612747,L.A. Lakers,Lakers,1626156,D'Angelo,Russell,0.0,4110.0,3,6.0,0.188
43,0022400062,1610612747,L.A. Lakers,Lakers,1629060,Rui,Hachimura,0.0,5370.0,5,5.0,0.143
...,...,...,...,...,...,...,...,...,...,...,...,...
35,0022400062,1610612747,L.A. Lakers,Lakers,2544,LeBron,James,25780.0,28800.0,3,-4.0,0.333
38,0022400062,1610612747,L.A. Lakers,Lakers,203076,Anthony,Davis,24510.0,28800.0,11,-1.0,0.375
47,0022400062,1610612747,L.A. Lakers,Lakers,1629060,Rui,Hachimura,27500.0,28800.0,0,-5.0,0.167
42,0022400062,1610612747,L.A. Lakers,Lakers,1626156,D'Angelo,Russell,21600.0,28800.0,2,-1.0,0.042


In [98]:
columns = ["Start_Time", "End_Time"] + [f"Player_{i}_{team}_{label}" for team in ["Home", "Away"] for i in range(1, 6) for label in ["ID", "Name"]]

In [99]:
lineup = pd.DataFrame(columns = columns)

In [ ]:
times = sorted(rotation['IN_TIME_REAL'].unique()) + [rotation['OUT_TIME_REAL'].max()]

In [103]:
# Starting lineup
current_lineup = rotation[rotation['IN_TIME_REAL'] == times[0]]

# Changing lineups during game
for t in times:

    next_time = times[times.index(t) + 1]

    # Players in lineup
    player_list = []
    for idx, player in current_lineup.iterrows():
        player_id = player['PERSON_ID']
        player_name = player['PLAYER_FIRST'] + " " + player['PLAYER_LAST']
        player_list.extend([player_id, player_name])

    new_row = [t, next_time] + player_list
    lineup = pd.concat([lineup, pd.DataFrame([new_row], columns = lineup.columns)], ignore_index=True)

    if t == times[-2]:
        break

    # Subbing out
    subbing_out_mask = current_lineup['OUT_TIME_REAL'] == next_time
    subbing_out_ids = current_lineup.loc[subbing_out_mask, 'PERSON_ID'].tolist()

    # Subbing in
    subbing_in = rotation[rotation['IN_TIME_REAL'] == next_time]
    assert len(subbing_out_ids) == len(subbing_in), f"Sub out/in mismatch at t={next_time}"

    # Perform subs
    current_lineup = current_lineup[~current_lineup['PERSON_ID'].isin(subbing_out_ids)]
    current_lineup = pd.concat([current_lineup, subbing_in], ignore_index=True)
    assert len(current_lineup) == 10, f"Lineup not complete at t={next_time}"

In [104]:
lineup

,Start_Time,End_Time,Player_1_Home_ID,Player_1_Home_Name,Player_2_Home_ID,Player_2_Home_Name,Player_3_Home_ID,Player_3_Home_Name,Player_4_Home_ID,Player_4_Home_Name,...,Player_1_Away_ID,Player_1_Away_Name,Player_2_Away_ID,Player_2_Away_Name,Player_3_Away_ID,Player_3_Away_Name,Player_4_Away_ID,Player_4_Away_Name,Player_5_Away_ID,Player_5_Away_Name
0,0.0,3160.0,201144,Mike Conley,203497,Rudy Gobert,203944,Julius Randle,1630162,Anthony Edwards,...,2544,LeBron James,203076,Anthony Davis,1626156,D'Angelo Russell,1629060,Rui Hachimura,1630559,Austin Reaves
1,3160.0,3540.0,201144,Mike Conley,203497,Rudy Gobert,203944,Julius Randle,1630162,Anthony Edwards,...,203076,Anthony Davis,1626156,D'Angelo Russell,1629060,Rui Hachimura,1630559,Austin Reaves,1631108,Max Christie
2,3540.0,4110.0,203497,Rudy Gobert,1630162,Anthony Edwards,1630183,Jaden McDaniels,203076,Anthony Davis,...,1629060,Rui Hachimura,1630559,Austin Reaves,1631108,Max Christie,1628978,Donte DiVincenzo,1629675,Naz Reid
3,4110.0,5370.0,203497,Rudy Gobert,1630162,Anthony Edwards,1630183,Jaden McDaniels,203076,Anthony Davis,...,1630559,Austin Reaves,1631108,Max Christie,1628978,Donte DiVincenzo,1629675,Naz Reid,1629216,Gabe Vincent
4,5370.0,5880.0,203497,Rudy Gobert,1630162,Anthony Edwards,203076,Anthony Davis,1631108,Max Christie,...,1629675,Naz Reid,1629216,Gabe Vincent,1629638,Nickeil Alexander-Walker,2544,LeBron James,1629637,Jaxson Hayes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,23640.0,24510.0,1629675,Naz Reid,2544,LeBron James,1629637,Jaxson Hayes,1626156,D'Angelo Russell,...,1630559,Austin Reaves,1628978,Donte DiVincenzo,1630162,Anthony Edwards,203944,Julius Randle,1629638,Nickeil Alexander-Walker
58,24510.0,25780.0,1629675,Naz Reid,1626156,D'Angelo Russell,1629060,Rui Hachimura,1630559,Austin Reaves,...,1630162,Anthony Edwards,203944,Julius Randle,1629638,Nickeil Alexander-Walker,203076,Anthony Davis,1629216,Gabe Vincent
59,25780.0,27500.0,1626156,D'Angelo Russell,1630559,Austin Reaves,1628978,Donte DiVincenzo,1630162,Anthony Edwards,...,1629638,Nickeil Alexander-Walker,203076,Anthony Davis,1629216,Gabe Vincent,203497,Rudy Gobert,2544,LeBron James
60,27500.0,28530.0,1626156,D'Angelo Russell,1630559,Austin Reaves,1628978,Donte DiVincenzo,1630162,Anthony Edwards,...,1629638,Nickeil Alexander-Walker,203076,Anthony Davis,203497,Rudy Gobert,2544,LeBron James,1629060,Rui Hachimura
